In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#TEST_IMAGE_DIR = "./dog-breed-identification/test/"
TRAIN_IMAGE_DIR = "./dog-breed-identification/train/"
LABELS = './dog-breed-identification/labels.csv'

data = pd.read_csv(LABELS)
class_names = data['breed'].unique()

print(data)

c:\Users\Danny\miniconda3\envs\tf\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


                                     id                     breed
0      000bec180eb18c7604dcecc8fe0dba07               boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                     dingo
2      001cdf01b096e06d78e9e5112d419397                  pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                  bluetick
4      0021f9ceb3235effd7fcde7f7538ed62          golden_retriever
...                                 ...                       ...
10217  ffd25009d635cfd16e793503ac5edef0                    borzoi
10218  ffd3f636f7f379c51ba3648a9ff8254f            dandie_dinmont
10219  ffe2ca6c940cddfee68fa3cc6c63213f                  airedale
10220  ffe5f6d8e2bff356e9482a80a6e29aac        miniature_pinscher
10221  fff43b07992508bc822f33d8ffd902ae  chesapeake_bay_retriever

[10222 rows x 2 columns]


In [2]:
#Data from the csv only contans the ids of the photos and not the photos itself
#Iterate through the data and check whether the photo is in test or train
#Then fetch it and store it in its proper variable
import tensorflow as tf
from os.path import join
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_images(image_dir, image_ids, target_size=(90, 90)):
    image_data = []
    for img_id in image_ids:
        img_path = join(image_dir, img_id + ".jpg")
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        preprocessed_img = preprocess_input(img_array) #I dont flatten the image here
        image_data.append(preprocessed_img)
    return np.array(image_data)

# Get image IDs for train and test
train_image_ids = data['id'].values
labels = data['breed'].values
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_image_ids, test_image_ids, train_labels, test_labels = train_test_split(train_image_ids, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels)

train_images = load_and_preprocess_images(TRAIN_IMAGE_DIR, train_image_ids)
test_images = load_and_preprocess_images(TRAIN_IMAGE_DIR, test_image_ids)

In [3]:
train_images = train_images / 250.0
test_images = test_images / 250.0

In [4]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

datagen.fit(train_images)

In [9]:
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization, Dropout

model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(90, 90, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(120, activation="softmax"),
])


In [10]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [11]:
batch_size = 64  # Choose a batch size that fits your memory constraints
epochs = 25

model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
          steps_per_epoch=len(train_images) // batch_size,
          epochs=epochs,
          validation_data=(test_images, test_labels))

Epoch 1/25
127/127 [==============================] - 11s 79ms/step - loss: 5.6018 - accuracy: 0.0141 - val_loss: 6.8943 - val_accuracy: 0.0147
Epoch 2/25
127/127 [==============================] - 10s 76ms/step - loss: 5.0860 - accuracy: 0.0245 - val_loss: 6.6162 - val_accuracy: 0.0073
Epoch 3/25
127/127 [==============================] - 10s 81ms/step - loss: 4.8649 - accuracy: 0.0274 - val_loss: 5.7095 - val_accuracy: 0.0176
Epoch 4/25
127/127 [==============================] - 10s 80ms/step - loss: 4.6990 - accuracy: 0.0322 - val_loss: 4.7673 - val_accuracy: 0.0308
Epoch 5/25
127/127 [==============================] - 10s 77ms/step - loss: 4.5686 - accuracy: 0.0393 - val_loss: 4.2877 - val_accuracy: 0.0616
Epoch 6/25
127/127 [==============================] - 10s 77ms/step - loss: 4.4944 - accuracy: 0.0423 - val_loss: 4.1905 - val_accuracy: 0.0836
Epoch 7/25
127/127 [==============================] - 10s 77ms/step - loss: 4.4012 - accuracy: 0.0465 - val_loss: 4.3657 - val_accuracy:

In [12]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

64/64 - 0s - loss: 3.5543 - accuracy: 0.1531 - 288ms/epoch - 4ms/step

Test accuracy: 0.15305623412132263
